In [22]:
import deepcut
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras
from pythainlp import word_vector
import re,string
from pythainlp import word_vector

In [2]:
def clean_msg(msg):
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    msg = re.sub(r'\d+', '',msg)
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    msg = re.sub(r'-','',msg)
    msg = re.sub(r'ฯ','',msg)
    msg = re.sub(r'ๆ','',msg)
    msg = re.sub(r'!@#$','',msg)
    msg = re.sub(r'[a-zA-Z]','',msg)
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

In [54]:
input_txt = open("corpus1.txt","r")
text = input_txt.read()
txt = clean_msg(text)
corpus = txt.split()
print(corpus)

['พยากรณ์อากาศ', 'ชั่วโมงข้างหน้า', 'บริเวณความกดอากาศสูงหรือมวลอากาศเย็นกำลังปานกลางจากประเทศจีนแผ่ปกคลุมประเทศไทยตอนบน', 'ประกอบกับมีหย่อมความกดอากาศต่ำปกคลุมบริเวณประเทศกัมพูชา', 'ลักษณะเช่นนี้ทำให้ประเทศไทยตอนบนมีฝนเกิดขึ้น', 'กับมีอากาศเย็น', 'ส่วนบริเวณยอดดอยและยอดภู', 'มีอากาศหนาว', 'สำหรับมรสุมตะวันออกเฉียงเหนือพัดปกคลุมอ่าวไทยและภาคใต้', 'ทำให้ภาคใต้มีฝนตกต่อเนื่อง', 'อนึ่ง', 'พายุโซนร้อน', 'อัสนี', 'พายุระดับ', 'บริเวณตะวันตกเฉียงใต้ของเกาะไต้หวัน', 'มีแนวโน้มที่จะอ่อนกำลังลง', 'และไม่มีผลกระทบต่อประเทศไทย', 'ในช่วงวันที่', 'พย', 'บริเวณความกดอากาศสูงหรือมวลอากาศเย็นกำลังปานกลางจากประเทศจีนปกคลุมประเทศไทยตอนบน', 'ประกอบกับมีหย่อมความกดอากาศต่ำปกคลุมบริเวณประเทศกัมพูชา', 'ลักษณะเช่นนี้ทำให้บริเวณประเทศไทยตอนบนมีอากาศเย็นกับมีฝนเล็กน้อยถึงปานกลาง', 'โดยเฉพาะบริเวณภาคตะวันออกเฉียงเหนือตอนล่างและภาคตะวันออก', 'สำหรับภาคใต้ตอนล่างมีฝนตกหนักบางแห่ง', 'เนื่องจากมรสุมตะวันออกเฉียงเหนือพัดปกคลุมอ่าวไทยและภาคใต้', 'ส่วนในช่วงวันที่', 'พย', 'บริเวณความกดอากาศสูงหรือมวลอากาศเย็นกำลังค่อน

In [55]:
def word_tokenize(corpus):
    x = 0
    words = []
    for i in corpus:
        list_word = deepcut.tokenize(i,custom_dict='custom_dict.txt') #ตัดคำ
        words.append(list_word)
        x += 1
    return words
# corpus=word_tokenize(corpus)
# x_test = word_tokenize(x_test)
words = word_tokenize(corpus)
print(words)
# print(len(words))

[['พยากรณ์อากาศ'], ['ชั่วโมง', 'ข้าง', 'หน้า'], ['บริเวณ', 'ความ', 'กด', 'อากาศ', 'สูง', 'หรือ', 'มวล', 'อากาศ', 'เย็น', 'กำลัง', 'ปานกลาง', 'จาก', 'ประเทศ', 'จีน', 'แผ่', 'ปกคลุม', 'ประเทศ', 'ไทย', 'ตอน', 'บน'], ['ประกอบ', 'กับ', 'มี', 'หย่อม', 'ความ', 'กด', 'อากาศ', 'ต่ำ', 'ปกคลุม', 'บริเวณ', 'ประเทศ', 'กัมพูชา'], ['ลักษณะ', 'เช่น', 'นี้', 'ทำ', 'ให้', 'ประเทศ', 'ไทย', 'ตอน', 'บน', 'มี', 'ฝน', 'เกิด', 'ขึ้น'], ['กับ', 'มี', 'อากาศ', 'เย็น'], ['ส่วน', 'บริเวณ', 'ยอด', 'ดอย', 'และ', 'ยอดภู'], ['มี', 'อากาศ', 'หนาว'], ['สำหรับ', 'มรสุม', 'ตะวัน', 'ออก', 'เฉียง', 'เหนือ', 'พัด', 'ปกคลุม', 'อ่าวไทย', 'และ', 'ภาค', 'ใต้'], ['ทำ', 'ให้', 'ภาค', 'ใต้', 'มี', 'ฝน', 'ตก', 'ต่อเนื่อง'], ['อนึ่ง'], ['พายุ', 'โซนร้อน'], ['อัสนี'], ['พายุ', 'ระดับ'], ['บริเวณ', 'ตะวัน', 'ตก', 'เฉียง', 'ใต้', 'ของ', 'เกาะ', 'ไต้หวัน'], ['มี', 'แนวโน้ม', 'ที่', 'จะ', 'อ่อน', 'กำลัง', 'ลง'], ['และ', 'ไม่', 'มี', 'ผล', 'กระทบ', 'ต่อ', 'ประเทศ', 'ไทย'], ['ใน', 'ช่วง', 'วัน', 'ที่'], ['พย'], ['บริเวณ', 'ความ', 'กด', 'อา

In [182]:
window_size = 1
def generate_data(corpus, window_size = 2):
    maxlen = window_size*2
    contexts = []
    labels   = []   
    for words in corpus:
        L = len(words)
        if len(words) < 2:
            contexts.append([''])
            labels.append(words[0])
        else:
            for index, word in enumerate(words):

                s = index - window_size
                e = index + window_size + 1

                contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
                labels.append(word)

    return contexts,labels
    #ฟังก์ชั้นสำหรับแบ่งข้อมูลให้อยู่ในรูป target and label

In [183]:
xtrain, ytrain = generate_data(words, window_size)

In [184]:
print(xtrain)
print(len(xtrain))

[[''], ['ข้าง'], ['ชั่วโมง', 'หน้า'], ['ข้าง'], ['ความ'], ['บริเวณ', 'กด'], ['ความ', 'อากาศ'], ['กด', 'สูง'], ['อากาศ', 'หรือ'], ['สูง', 'มวล'], ['หรือ', 'อากาศ'], ['มวล', 'เย็น'], ['อากาศ', 'กำลัง'], ['เย็น', 'ปานกลาง'], ['กำลัง', 'จาก'], ['ปานกลาง', 'ประเทศ'], ['จาก', 'จีน'], ['ประเทศ', 'แผ่'], ['จีน', 'ปกคลุม'], ['แผ่', 'ประเทศ'], ['ปกคลุม', 'ไทย'], ['ประเทศ', 'ตอน'], ['ไทย', 'บน'], ['ตอน'], ['กับ'], ['ประกอบ', 'มี'], ['กับ', 'หย่อม'], ['มี', 'ความ'], ['หย่อม', 'กด'], ['ความ', 'อากาศ'], ['กด', 'ต่ำ'], ['อากาศ', 'ปกคลุม'], ['ต่ำ', 'บริเวณ'], ['ปกคลุม', 'ประเทศ'], ['บริเวณ', 'กัมพูชา'], ['ประเทศ'], ['เช่น'], ['ลักษณะ', 'นี้'], ['เช่น', 'ทำ'], ['นี้', 'ให้'], ['ทำ', 'ประเทศ'], ['ให้', 'ไทย'], ['ประเทศ', 'ตอน'], ['ไทย', 'บน'], ['ตอน', 'มี'], ['บน', 'ฝน'], ['มี', 'เกิด'], ['ฝน', 'ขึ้น'], ['เกิด'], ['มี'], ['กับ', 'อากาศ'], ['มี', 'เย็น'], ['อากาศ'], ['บริเวณ'], ['ส่วน', 'ยอด'], ['บริเวณ', 'ดอย'], ['ยอด', 'และ'], ['ดอย', 'ยอดภู'], ['และ'], ['อากาศ'], ['มี', 'หนาว'], ['อากาศ'], ['มรสุม'], 

In [185]:
print(ytrain)
print(len(ytrain))

['พยากรณ์อากาศ', 'ชั่วโมง', 'ข้าง', 'หน้า', 'บริเวณ', 'ความ', 'กด', 'อากาศ', 'สูง', 'หรือ', 'มวล', 'อากาศ', 'เย็น', 'กำลัง', 'ปานกลาง', 'จาก', 'ประเทศ', 'จีน', 'แผ่', 'ปกคลุม', 'ประเทศ', 'ไทย', 'ตอน', 'บน', 'ประกอบ', 'กับ', 'มี', 'หย่อม', 'ความ', 'กด', 'อากาศ', 'ต่ำ', 'ปกคลุม', 'บริเวณ', 'ประเทศ', 'กัมพูชา', 'ลักษณะ', 'เช่น', 'นี้', 'ทำ', 'ให้', 'ประเทศ', 'ไทย', 'ตอน', 'บน', 'มี', 'ฝน', 'เกิด', 'ขึ้น', 'กับ', 'มี', 'อากาศ', 'เย็น', 'ส่วน', 'บริเวณ', 'ยอด', 'ดอย', 'และ', 'ยอดภู', 'มี', 'อากาศ', 'หนาว', 'สำหรับ', 'มรสุม', 'ตะวัน', 'ออก', 'เฉียง', 'เหนือ', 'พัด', 'ปกคลุม', 'อ่าวไทย', 'และ', 'ภาค', 'ใต้', 'ทำ', 'ให้', 'ภาค', 'ใต้', 'มี', 'ฝน', 'ตก', 'ต่อเนื่อง', 'อนึ่ง', 'พายุ', 'โซนร้อน', 'อัสนี', 'พายุ', 'ระดับ', 'บริเวณ', 'ตะวัน', 'ตก', 'เฉียง', 'ใต้', 'ของ', 'เกาะ', 'ไต้หวัน', 'มี', 'แนวโน้ม', 'ที่', 'จะ', 'อ่อน', 'กำลัง', 'ลง', 'และ', 'ไม่', 'มี', 'ผล', 'กระทบ', 'ต่อ', 'ประเทศ', 'ไทย', 'ใน', 'ช่วง', 'วัน', 'ที่', 'พย', 'บริเวณ', 'ความ', 'กด', 'อากาศ', 'สูง', 'หรือ', 'มวล', 'อากาศ', 'เ

In [186]:
w2v = word_vector.get_model()

In [187]:
vecter = []
vocab_ = []
index = 0
for word in w2v.index2word:
    vocab_.append(word)
    vecter.append(w2v[word])
print(len(vocab_))
print(len(vecter))

51358
51358


In [188]:
def sent2idx(x):
    xidx = []
    for w in x:
#         print(w)
        if w in vocab_:
            xidx.append(vocab_.index(w))
        else:
            vocab_.append(w)
#             np.vstack((vecter,np.random.rand(*vecter[0].shape)))
            vecter.append(np.random.rand(*vecter[0].shape))
            xidx.append(vocab_.index(w))
    return np.array(xidx)

In [189]:
# print(vecter[1])
print(len(vocab_))
print(len(vecter))

51358
51358


In [190]:
def word_seq(words):
    Xtrain_idx = []
    maxlen = 0
    for x in words:
        Xtrain_idx.append(sent2idx(x))
        if len(Xtrain_idx[-1]) > maxlen:
            maxlen = len(Xtrain_idx[-1])
#         print(len(Xtrain_idx[-1]))
    return Xtrain_idx

In [193]:
vocab_ = [''] + vocab_
vecter = np.vstack((np.random.rand(*vecter[0].shape), vecter))

In [194]:
Ytrain = np.array(sent2idx(ytrain))
print(len(vocab_))
print(len(vecter))

51367
51367


In [203]:
Xtrain = np.array(word_seq(xtrain))
Xtrain = sequence.pad_sequences(np.array(Xtrain),padding='pre')
print(len(Xtrain))
print(Xtrain.shape)

810
(810, 2)


In [202]:
print(Ytrain.shape)

(810,)
